In [2]:
import pandas as pd
import json
import sqlite3
from datetime import datetime

In [1]:
# CONFIGURACIÓN
INPUT_FILE = '../data/raw/kavak_raw_data.jsonl'
DB_NAME = '../kavak_market.db'

In [3]:
data = []

with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            continue
    print(f"📥 Registros crudos leídos: {len(data)}")

📥 Registros crudos leídos: 1550


In [4]:
processed_data = []

for entry in data:
    try:
        raw = entry['raw_data']
        car = next((item for item in raw.get('@graph',[]) if item.get('@type') == 'Car'), None)
        
        if car:
            # Extracción segura con .get() para evitar errores si falta un campo
                record = {
                    'id_interno': entry['source_url'].split('/')[-1], # Generamos ID único desde la URL
                    'marca': car.get('brand', {}).get('name', 'Desconocido'),
                    'modelo': car.get('model', 'Desconocido'),
                    'version': car.get('vehicleConfiguration', 'N/A'),
                    'anio': car.get('vehicleModelDate', 0),
                    'precio_mxn': car.get('offers', {}).get('price', 0),
                    'km': car.get('mileageFromOdometer', {}).get('value', 0),
                    'transmision': car.get('vehicleTransmission', 'N/A'),
                    'ciudad': 'Mexico', # Kavak suele centralizar, por ahora hardcodeamos o extraemos si hay info
                    'url': entry['source_url'],
                    'fecha_extraccion': datetime.fromtimestamp(entry['extracted_at']).strftime('%Y-%m-%d')
                }
                processed_data.append(record)
    except Exception as e:
        continue

df = pd.DataFrame(processed_data)
print(f"📊 Registros procesados: {len(df)}")

📊 Registros procesados: 1550


In [5]:
# --- LIMPIEZA DE TIPOS (TYPE CASTING) ---
# Convertimos a numérico, los errores se vuelven NaN (Not a Number)
df['anio'] = pd.to_numeric(df['anio'], errors='coerce').fillna(0).astype(int)
df['km'] = pd.to_numeric(df['km'], errors='coerce').fillna(0).astype(int)
df['precio_mxn'] = pd.to_numeric(df['precio_mxn'], errors='coerce').fillna(0).astype(int)

# Filtrar basura (Autos con precio 0 o año 0)
df = df[df['precio_mxn'] > 0]

print(f"   ✨ Registros limpios listos para SQL: {len(df)}")
print(df.dtypes) # Para que verifiques que ahora sí son int32/int64

   ✨ Registros limpios listos para SQL: 1550
id_interno          object
marca               object
modelo              object
version             object
anio                 int64
precio_mxn           int64
km                   int64
transmision         object
ciudad              object
url                 object
fecha_extraccion    object
dtype: object


In [7]:
# 3. CARGAR (Load to SQL)
conn = sqlite3.connect(DB_NAME)

# Guardamos en tabla 'autos'. 
# if_exists='replace': Borra la tabla y la crea de nuevo (útil para desarrollo)
# index=False: No guardar el índice numérico de Pandas
df.to_sql('autos', conn, if_exists='replace', index=False)

conn.close()
print(f"   💾 Datos guardados exitosamente en {DB_NAME}")

   💾 Datos guardados exitosamente en kavak_market.db
